In [ ]:
# configurando o SMTP

import smtplib

smtpObj1 = smtplib.SMTP('smtp.gmail.com', 587) # TLS
smtpObj2 = smtplib.SMTP_SSL('smtp.gmail.com', 465) # SSL

type(smtpObj)

In [2]:
# enviando a menssagem de 'Hello' do SMTP

import smtplib

smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
smtpObj.ehlo() # o primeiro item da tupla deve ser o inteiro 250 (que é o codigo de sucesso do SMTP)

# para porta 587 (TLS) é obrigatorio iniciar a criptografia com o metódo starttls(), conforme abaixo.:

smtpObj.starttls() # 220 no retorno indica que o servidor está pronto

# logar no e-mail

smtpObj.login('pda.ambrosio@gmail.com', input()) # criar senha de APP, 235 indica que a conexão foi bem sucedida

(235, b'2.7.0 Accepted')

In [3]:
# enviando um e-mail

smtpObj.sendmail('pda.ambrosio@gmail.com', 'juliana.a.a.s2705@gmail.com', 'Subject: So long.\nDear Juliana, so long and thanks for all the fish. Sincerely, Paul')  # o metodo sendmail possui três argumentos, o primeito é o e-mail do remetente 'from', o segundo é o destinatario 'to' e o terceiro uma string com o conteudo do e-mail que deve começar com 'Subject:' para o assunto e '\n' para o conteudo do e-mail

# este deve retornar um dicionario vazio, indicando que o e-mail foi enviado com sucesso



{}

In [4]:
# desconectando do servidor SMTP

smtpObj.quit() # o valor 221 indica que a conexão foi encerrada

(451, b'4.4.2 Timeout - closing connection. o125sm642559qke.56 - gsmtp')

In [1]:
# IMAP - Internet Message Access Protocol

import imapclient

# conectando ao servidor
imapConnect = imapclient.IMAPClient('imap.gmail.com', ssl=True)

# fazendo login
imapConnect.login('pda.ambrosio@gmail.com', input())

# procurando e-mails
import pprint

pprint.pprint(imapConnect.list_folders()) # lista os diretorios no seu INBOX
imapConnect.select_folder('INBOX', readonly=True) # selecionar uma pasta

# realizando pesquisa
imapConnect.search(['ALL']) # retorna todas as menssagens da pasta
imapConnect.search(['ON 05-Jul-2020']) # retorna todas as mensagens enviadas em 05 de Julho de 2020
imapConnect.search(['SINCE 01-Jan-2020', 'BEFORE 01-Feb-2020', 'UNSEEN']) # retorna todas as menssagens enviadas em 01 de Janeiro de 2020 que não tenham sido lidas (seria de 01 Janeiro de 2020 até 01 de Fevereiro de 2020, sem incluir o 01 de Fereiro)
imapConnect.search(['SINCE 01-Jan-2020', 'FROM exemple@example.com']) # retorna todas as mensagens de "exemple@example.com" enviadas desde o inicio de Janeiro de 2020
imapConnect.search(['SINCE 01-Jan-2020', 'NOT FROM exemple@example.com']) # retorna todas as mensagens enviadas por todos, exceto por "exemple@example.com", desde Janeiro de 2020
imapConnect.search(['OR FROM exemple@example.com FROM juliana@example.com']) # retorna todas as mensagens enviadas por "exemple@example.com" ou por "juliana@example.com"
imapConnect.search(['FROM exemple@example.com', 'FROM juliana@example.com']) # jamais irá retornar as mensagens vistos que só pode haver um endereço from

# o metodo search irá retornar UIDs

# limites de tamanho (limite padrão 10.000 bytes)
import imaplib

imaplib._MAXLINE = 10000000

# para gmail existe um metodo search mais sofisticado
imapConnect.gmail_search('meaning of life')

# buscando e-mails e marcando como não lidos

# e-mails de juliana.a.a.s@gmail.com que irá gerar uma lista de UIDs
UIDs = imapConnect.search(['FROM juliana.a.a.s@gmail.com'])
# lendo mensagens
rawMessages = imapConnect.fetch(UIDs, ['BODY[]'])
import pprint
pprint.pprint(rawMessages)

# para que os e-mail acessados sejam marcados como lidos
imapConnect.select_folder('INBOX',readonly=False)

[((b'\\HasNoChildren',), b'/', 'Amigos-Júndiai'),
 ((b'\\HasNoChildren',), b'/', 'Amigos-SP'),
 ((b'\\HasNoChildren',), b'/', 'Arquivar'),
 ((b'\\HasNoChildren',), b'/', 'Assosiação'),
 ((b'\\HasChildren',), b'/', 'Banco Santander'),
 ((b'\\HasNoChildren',), b'/', 'Banco Santander/Pagamentos Santander'),
 ((b'\\HasNoChildren',), b'/', 'Cadastros Variados'),
 ((b'\\HasNoChildren',), b'/', 'Certificados'),
 ((b'\\HasNoChildren',), b'/', 'Clube do Hardware'),
 ((b'\\HasNoChildren',), b'/', 'Compras On-Line'),
 ((b'\\HasNoChildren',), b'/', 'Compras WAZ'),
 ((b'\\HasNoChildren',), b'/', 'Compras no Exterior'),
 ((b'\\HasNoChildren',), b'/', 'Contatos Profissionais'),
 ((b'\\HasNoChildren',), b'/', 'Cursos On-line'),
 ((b'\\HasNoChildren',), b'/', 'Deleted Items'),
 ((b'\\HasNoChildren',), b'/', 'Fundação Bradesco'),
 ((b'\\HasNoChildren',), b'/', 'Gmail - Drive'),
 ((b'\\HasNoChildren',), b'/', 'INBOX'),
 ((b'\\HasNoChildren',), b'/', 'IR'),
 ((b'\\HasNoChildren',), b'/', 'Ingressos.com'),

In [2]:
# obtendo mensagens puras
import pyzmail # faz o parse do e-mail, erro ao instalar via PIP

message = pyzmail.PyzMessage.factory(rawMessages[UID]['BODY[]'])
message.get_subject()
message.get_addresses('from')
message.get_addresses('to')
message.get_addresses('cc')
message.get_addresses('bcc')

message.text_part != None
message.text_part.get_payload().decode(message.text_part.charset) # caso a linha de cima seja True

message.html_part != None
message.get_payload().decode(message.html_part.charset)

ModuleNotFoundError: No module named 'pyzmail'

In [ ]:
# apagando e-mails
imapConnect.select_folder('INBOX', readonly=False)
UIDs = imapConnect.search(['ON 01-Jan-2020'])

imapConnect.delete_messages(UIDs)
imapConnect.expunge() # se for gmail, não será necessário

In [ ]:
# desconectando do servidor IMAP

imapConnect.logout()

In [17]:
# enviando SMS com Twilio
# após ter criado sua conta e instalado o modulo...

from twilio.rest import Client

accountSID = 'ACd38918e78df8f80dac0ef3cfe303ba93'
authToken = '479f3ae04476660358cc5c1b6f720626'

twilioCli = Client(accountSID, authToken)
myTwilioNumber = "+12056977443"
myCellPhone = "+5511972424687"

message = twilioCli.messages.create(body='Mr. Watson - Como here - I want to see you.', from_=myTwilioNumber, to=myCellPhone)

In [22]:
print(message.status)
print(message.date_created)
print(message.date_sent == None)
print(message.sid)

queued
2020-12-01 01:32:12+00:00
True
SM3a02bbdd27ba48be99d40ceb677305d6


In [29]:
updatedMessage = twilioCli.messages.get(message.sid)
print(updatedMessage)

<Twilio.Api.V2010.MessageContext account_sid=ACd38918e78df8f80dac0ef3cfe303ba93 sid=SM3a02bbdd27ba48be99d40ceb677305d6>


In [3]:
# utilizando o textMyself.py

import cap16_textMyself

